In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from astropy import time as astropytime
from ctapipe.io import event_source, EventSeeker
from ctapipe.io.nectarcameventsource import NectarCAMEventSource
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.image import tailcuts_clean, dilate, hillas_parameters, HillasParameterizationError

In [ ]:
path = 'obs/NectarCAM.Run1388.0001.fits.fz'
cmap = 'gnuplot2'

In [ ]:
reader = NectarCAMEventSource(input_url=path)
# run 1388 has interleaved pedestals
seeker = EventSeeker(reader)
camera = seeker[0].inst.subarray.tel[0].camera

## Look at waveform image for a particular event

In [ ]:
evt = seeker[25]
import time
from IPython import display

adcsum = evt.r0.tel[0].waveform[0].sum(axis=1)

for i in range(len(evt.r0.tel[0].waveform[0])-1):
    image = evt.r0.tel[0].waveform[0].T[i]
    plt.clf()

    fig = plt.figure(figsize=(8,4))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    disp2= CameraDisplay(geometry=camera, image=adcsum, ax=ax1, title='Sum ADC', cmap=cmap)
    #disp2.cmap = cmap
    #disp2.add_colorbar()

    disp = CameraDisplay(geometry=camera, image=image, ax=ax2, title='Waveform (ADC), T={} ns'.format(i), cmap=cmap)
    #disp.cmap = cmap
    disp.add_colorbar()

    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(0.2)

## Or look at an integrated charge image for a particular event

In [ ]:
image = seeker[0].r0.tel[0].waveform.sum(axis=2)
disp = CameraDisplay(geometry=camera, image=image[0], cmap=cmap)

In [ ]:
run_start = astropytime.Time(seeker[0].nectarcam.tel[0].svc.date, format='unix').iso
print('Run started at {}'.format(run_start))

## Extract interleaved pedestals

In [ ]:
# Evaluate pedestal from interleaved pedestals from same input run
max_events = 500
read_ped = NectarCAMEventSource(input_url=path)
peds = []
for i, ev in enumerate(read_ped):
    if len(peds) > max_events:
        break
    if ev.r0.tel[0].trigger_type == 1:
        # print('Event {}, trigger type {}'.format(i,ev.r0.tel[0].trigger_type))
        wfs = ev.r0.tel[0].waveform
        wfs_hi = wfs[0].sum(axis=1)
        peds.append(wfs_hi)
peds = np.array(peds)
peds = peds.mean(axis=0)

In [ ]:
disp = CameraDisplay(geometry=camera, image=peds, cmap=cmap)
disp.cmap = cmap
disp.add_colorbar()

In [ ]:
plt.plot(peds)

## Calibration

In [ ]:
adc_to_pe = 58.
evt = next(iter(seeker))
print('Event {}, trigger type {}'.format(evt.r0.event_id,
                                         evt.r0.tel[0].trigger_type))
if evt.r0.tel[0].trigger_type == 0:
    raw = evt.r0.tel[0].waveform[0].sum(axis=1)
    charges = ((raw - peds)/adc_to_pe)
disp = CameraDisplay(geometry=camera, image=charges, cmap='gnuplot2')
disp.cmap = cmap
disp.add_colorbar()

## Hillas cleaning

In [ ]:
cleanmask = tailcuts_clean(camera, charges,
                           picture_thresh=10,
                           boundary_thresh=5,
                           min_number_picture_neighbors=3)
charges[~cleanmask] = 0
try:
    hillas_param = hillas_parameters(camera, charges)
    disp = CameraDisplay(geometry=camera, image=charges, cmap='gnuplot2')
    disp.cmap = cmap
    disp.add_colorbar()
    disp.overlay_moments(hillas_param, with_label=False,
                     color='red', alpha=0.7,
                     linewidth=2, linestyle='dashed')
    disp.highlight_pixels(cleanmask, color='white', alpha=0.3, linewidth=2)
    print(hillas_param)
except HillasParameterizationError:
    pass
print('Cleaned image: charge = {} pe'.format(charges.sum()))

## Loop over events

In [ ]:
import time
from IPython import display
for i, evt in enumerate(reader):
    if evt.r0.tel[0].trigger_type == 0:
        raw = evt.r0.tel[0].waveform[0].sum(axis=1)
        charges = ((raw - peds)/adc_to_pe)
        cleanmask = tailcuts_clean(camera, charges,
                                   picture_thresh=10,
                                   boundary_thresh=5,
                                   min_number_picture_neighbors=3)
        charges[~cleanmask] = 0

        plt.clf()
        disp = CameraDisplay(geometry=camera, image=charges, cmap='gnuplot2')
        disp.cmap = cmap
        disp.add_colorbar()
        try:
            hillas_param = hillas_parameters(camera, charges)
            disp.overlay_moments(hillas_param, with_label=False,
                                 color='red', alpha=0.7,
                                 linewidth=2, linestyle='dashed')
            disp.highlight_pixels(cleanmask, color='white', alpha=0.3, linewidth=2)
        except HillasParameterizationError:
            pass
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(0.2)